In [4]:
import numpy as np
import math
import pandas as pd
import matplotlib as plt
import datetime
%matplotlib inline

## Getting User Data

* User fills out survey, ranking the features 1-6
* User also provides constraints, (e.g. number of bedrooms, number of bathrooms, max rental price)

## Initial Recommendation

* Filter listings by the constraints, ONLY show those that satisfy the constraints.
    * Do **NOT** include those that are already shown or those that are disliked.
* Shift all the norm values up by 10 to get rid of negative values
* Matrix multiply user rankings with the new norm values to create a new score for that user's weights
* Show the top listing
    * If **LIKE:**
        * Add to _Likes_ dictionary, add to _Shown_ dictionary
        * Prompt user _"Do you want to see more like these?"_
            * If **YES:**
                * Move into the next code chunk, show similar listings, return to the top of the loop.
            * If **NO:**
                * Encounter finished, exit.
    * If **DISLIKE:**
        * Add to _Dislikes_ dictionary, add to _Shown_ dictionary
        * Add (1-2) similar to this listing to the _Dislikes_ dictionary
        * Generate a new initial recommendation, return to the top of the loop.

In [52]:
# def initial_rec(address, preferences):
    

## Similar Listings after Initial Recommendation (Content Based Recommender)
https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

In [21]:
subset = pd.read_csv("scored_housing/full_housing.csv")
subset

,Unnamed: 0,Unnamed: 0.1,Listing ID,Location,Lat_Long,Monthly Rental Price,Bedrooms,Bathrooms,Square Footage,Latitude,...,num_restaurants,num_subways,num_trees,grocers_normalized,nightlife_normalized,noise_normalized,restaurants_normalized,subways_normalized,trees_normalized,id
0,0,0,3021099,211 North End Avenue #12A,"['40.71628546,-74.01546103']",4600,1.0,1.0,860.0,40.716285,...,26,0,430,-1.116985,-0.961782,-1.153283,-0.909475,-0.765301,0.183035,3021099
1,1,1,3020988,150 East 39th Street #601,"['40.74890137,-73.97660065']",2650,0.0,1.0,0.0,40.748901,...,108,3,514,0.304876,-0.137721,-0.217637,0.343502,-0.321111,0.661842,3020988
2,2,2,3026635,257 West 136th Street #4AA,"['40.81700815,-73.94508554']",2195,2.0,1.0,0.0,40.817008,...,12,6,561,0.399667,-0.373167,0.366263,-1.123398,0.123079,0.929746,3026635
3,3,3,3026633,272 West 139th Street #3A,"['40.81876179,-73.9446072']",2650,2.0,1.0,0.0,40.818762,...,13,2,554,0.020504,-0.608613,0.795394,-1.108118,-0.469174,0.889845,3026633
4,4,4,3026626,408 West 34th Street #6E,"['40.75329123,-73.9971017']",1975,0.0,1.0,0.0,40.753291,...,71,0,210,-0.832613,-0.490890,0.387368,-0.221866,-0.765301,-1.070983,3026626
5,5,5,3026621,47 West 88th Street #4A,"['40.78749503,-73.96993322']",2350,1.0,1.0,0.0,40.787495,...,16,5,583,-0.832613,-1.079505,-0.646768,-1.062277,-0.024985,1.055148,3026621
6,6,6,3026616,2147 Second Avenue #5A,"['40.79360168,-73.94066629']",2565,2.0,1.0,0.0,40.793602,...,34,0,289,0.684040,-0.961782,-0.203567,-0.787234,-0.765301,-0.620676,3026616
7,7,7,3026611,516 East 78th Street #4A,"['40.77019882,-73.9496994']",2200,0.0,1.0,0.0,40.770199,...,21,0,399,-0.643032,-1.079505,-0.688977,-0.985876,-0.765301,0.006333,3026611
8,8,8,3026608,2 Clinton Street #6C,"['40.72117927,-73.98382714']",4300,3.0,1.0,0.0,40.721179,...,99,0,485,1.063203,1.628123,1.442608,0.205980,-0.765301,0.496540,3026608
9,9,9,3026607,279 East 44th Street #7H,"['40.75152284,-73.97138232']",2700,0.0,1.0,550.0,40.751523,...,120,0,466,-0.453450,-0.373167,-0.696012,0.526864,-0.765301,0.388238,3026607


In [22]:
vals = subset.iloc[:, 18:24] + 10
vals.head()

,grocers_normalized,nightlife_normalized,noise_normalized,restaurants_normalized,subways_normalized,trees_normalized
0,8.883015,9.038218,8.846717,9.090525,9.234699,10.183035
1,10.304876,9.862279,9.782363,10.343502,9.678889,10.661842
2,10.399667,9.626833,10.366263,8.876602,10.123079,10.929746
3,10.020504,9.391387,10.795394,8.891882,9.530826,10.889845
4,9.167387,9.509110,10.387368,9.778134,9.234699,8.929017


In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
cosine_sim = cosine_similarity(vals)
cosine_sim

array([[1.        , 0.99924818, 0.99820424, ..., 0.99954346, 0.999533  ,
        0.99395805],
       [0.99924818, 1.        , 0.99769486, ..., 0.99971877, 0.99953334,
        0.99297043],
       [0.99820424, 0.99769486, 1.        , ..., 0.99861665, 0.99835162,
        0.99603691],
       ...,
       [0.99954346, 0.99971877, 0.99861665, ..., 1.        , 0.99990587,
        0.9947071 ],
       [0.999533  , 0.99953334, 0.99835162, ..., 0.99990587, 1.        ,
        0.99475874],
       [0.99395805, 0.99297043, 0.99603691, ..., 0.9947071 , 0.99475874,
        1.        ]])

In [24]:
indices = pd.Series(subset["Location"])

# Defining the function that takes in listing name
# as input and returns the top 10 recommended listings
def recommendations(address, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended listings
    recommended_addresses = []
    
    # gettin the index of the listing that matches the title
    idx = indices[indices == address].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar listings
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching listings
    for i in top_10_indexes:
        recommended_addresses.append(list(vals.index)[i])
        
    return recommended_addresses

In [36]:
[indices[x] for x in recommendations("211 North End Avenue #12A")]

['325 North End Avenue #16L',
 '134 East 36th Street #1',
 '1080 Amsterdam Avenue #1406',
 '228 West 71st Street #9C',
 '400 West 113th Street #1220',
 '510 Amsterdam Avenue #3A',
 '142 West 86th Street #5A',
 '331 West 76th Street #2A',
 '331 West End Avenue #1A',
 '154 West 70th Street #3R']

## Visualizations

When users receive a recommendation, they should be able to see visually **where the listing is on a map.**
Ideally, they should be able to click on a listing and a pop-up should appear with the following:
* Apartment name
* Beds
* Bathrooms
* Square footage (if available)
* Price
* Some generic description of why we recommended
    * _You prioritized **X**, there are **Y** instances of this within a 0.25 mile radius of this listing._

Users should also be able to see things they marked as _**Liked**_ in a similar visualization as mentioned above.